In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchvision import datasets,transforms,models
from torch.autograd import Variable

In [2]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.Compose([ 
                                  transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ]))
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.Compose([
                                   transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ]))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# load data
trainLoader = torch.utils.data.DataLoader(trainset,batch_size=8,shuffle=True)
testLoader = torch.utils.data.DataLoader(testset,batch_size=8,shuffle=True)

In [4]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
# import matplotlib.pyplot as plt
# import numpy as np

# # functions to show an image


# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()


# # get some random training images
# dataiter = iter(train_loader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [6]:
# Lenet5
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model,self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self,x):
        x = self.relu(self.pool(self.conv1(x)))
        x = self.relu(self.pool(self.conv2(x)))
        x = x.view(-1, 16*5*5)#reshape
        
        # =====
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
        
        

In [7]:
net = CNN_model()
net.cuda()

CNN_model(
  (relu): ReLU()
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
device = torch.device("cuda")
# loss_fn = nn.NLLLoss() # One way to calculate loss
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.001,momentum=0.9)




In [11]:
# Train
for epoch in range(3):
    running_loss = 0.0

    for times, data in enumerate(trainLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        
        print(inputs.size())
        print(labels)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if times % 100 == 99 or times+1 == len(trainLoader):
            print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, 3, times+1, len(trainLoader), running_loss/2000))

print('Finished Training')

torch.Size([8, 3, 32, 32])
tensor([1, 9, 5, 4, 2, 2, 9, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 7, 6, 3, 7, 0, 7, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 0, 3, 3, 8, 6, 0, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 8, 4, 0, 1, 8, 8, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 5, 5, 4, 2, 8, 9, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 0, 1, 8, 4, 1, 1, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 9, 6, 4, 8, 6, 5, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 6, 1, 9, 9, 4, 6, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 2, 2, 8, 4, 4, 7, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 4, 7, 8, 8, 0, 4, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 5, 6, 1, 5, 5, 2, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 2, 8, 0, 6, 5, 5, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 1, 3, 0, 4, 7, 3, 3], device='cuda:0')

tensor([7, 8, 7, 1, 3, 2, 2, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 8, 3, 3, 4, 4, 3, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 1, 2, 5, 7, 0, 8, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 3, 2, 8, 6, 9, 1, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 7, 6, 7, 4, 9, 2, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 8, 1, 8, 7, 3, 1, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 3, 5, 2, 5, 9, 5, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 7, 4, 4, 1, 8, 7, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 0, 4, 4, 7, 7, 1, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 6, 1, 5, 8, 6, 9, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 3, 6, 7, 4, 3, 9, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 0, 0, 3, 6, 3, 2, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 5, 7, 6, 2, 1, 0, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])

tensor([9, 1, 1, 2, 9, 7, 0, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 8, 3, 7, 3, 5, 8, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 6, 5, 6, 0, 6, 2, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 8, 6, 9, 4, 6, 0, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 8, 6, 5, 6, 6, 7, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 9, 8, 2, 3, 1, 2, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 9, 6, 7, 6, 8, 0, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 5, 4, 5, 6, 2, 6, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 7, 4, 9, 7, 7, 7, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 9, 2, 5, 9, 8, 7, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 5, 5, 8, 5, 3, 2, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 8, 0, 0, 5, 5, 1, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 4, 4, 1, 7, 0, 2, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])

torch.Size([8, 3, 32, 32])
tensor([0, 7, 8, 6, 7, 3, 2, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 9, 0, 5, 6, 1, 5, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 1, 4, 8, 5, 7, 3, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 5, 8, 0, 4, 2, 6, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 9, 4, 7, 2, 2, 6, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 2, 6, 6, 8, 9, 1, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 3, 1, 7, 2, 1, 7, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 4, 5, 9, 0, 7, 8, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 3, 4, 6, 9, 1, 0, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 1, 5, 2, 1, 2, 3, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 6, 8, 3, 3, 4, 5, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 6, 8, 9, 3, 8, 0, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 4, 7, 5, 5, 1, 0, 5], device='cuda:0')

tensor([7, 4, 7, 8, 2, 8, 8, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 7, 1, 6, 8, 6, 8, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 1, 1, 8, 4, 2, 8, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 0, 3, 1, 7, 9, 9, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 6, 9, 9, 7, 0, 2, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 8, 7, 0, 3, 2, 4, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 9, 7, 4, 7, 9, 7, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 4, 7, 6, 2, 2, 2, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 9, 0, 6, 6, 7, 5, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 0, 2, 8, 7, 9, 4, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 0, 9, 1, 3, 7, 9, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 1, 0, 9, 1, 8, 8, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 0, 6, 0, 3, 5, 4, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])

...

tensor([5, 2, 1, 6, 5, 0, 1, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 0, 9, 7, 1, 2, 1, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 2, 9, 0, 3, 6, 1, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 7, 8, 0, 4, 6, 1, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 5, 1, 4, 4, 0, 4, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 3, 5, 7, 2, 1, 6, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 4, 5, 1, 2, 8, 0, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 0, 5, 3, 6, 4, 0, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 2, 4, 6, 1, 5, 9, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 6, 8, 3, 5, 0, 9, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 8, 0, 1, 2, 5, 1, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 1, 4, 3, 8, 4, 8, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 7, 6, 6, 1, 2, 0, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])

tensor([3, 3, 0, 6, 4, 5, 1, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([1, 3, 5, 4, 6, 8, 8, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 3, 7, 6, 3, 9, 3, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 6, 0, 6, 3, 4, 9, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 0, 6, 0, 5, 7, 2, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 7, 1, 4, 2, 1, 5, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 9, 2, 5, 7, 4, 5, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 1, 6, 1, 3, 8, 4, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 4, 5, 1, 6, 4, 3, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 6, 7, 7, 9, 5, 3, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 5, 3, 4, 1, 0, 2, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 6, 4, 6, 2, 7, 3, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 7, 5, 6, 3, 7, 1, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])

tensor([5, 7, 6, 0, 4, 4, 1, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 1, 9, 6, 1, 8, 7, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 5, 0, 8, 5, 8, 3, 9], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 6, 3, 7, 5, 3, 1, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 4, 2, 9, 9, 6, 2, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 5, 7, 2, 9, 8, 0, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 1, 3, 8, 5, 2, 1, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 3, 2, 9, 9, 4, 2, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 6, 8, 0, 6, 9, 7, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 2, 6, 6, 0, 7, 5, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 5, 9, 5, 6, 2, 2, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 6, 1, 8, 0, 4, 5, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 5, 8, 5, 6, 6, 4, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])

tensor([3, 3, 9, 2, 6, 1, 0, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 7, 4, 2, 8, 8, 2, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 4, 8, 7, 6, 5, 6, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 4, 3, 8, 0, 0, 4, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 8, 8, 4, 0, 3, 7, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 4, 3, 6, 8, 1, 7, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 4, 3, 7, 0, 8, 9, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 5, 5, 2, 9, 7, 3, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 4, 6, 0, 4, 0, 9, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([6, 7, 3, 8, 2, 3, 3, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([0, 4, 4, 9, 2, 6, 7, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([8, 6, 1, 9, 4, 7, 2, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 6, 0, 4, 4, 9, 1, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])

tensor([3, 6, 5, 7, 0, 4, 1, 8], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 4, 3, 1, 0, 6, 5, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 9, 9, 5, 2, 3, 6, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 0, 4, 7, 9, 9, 1, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 6, 9, 7, 3, 6, 9, 7], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 2, 0, 6, 4, 1, 9, 4], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([7, 9, 9, 9, 5, 8, 9, 6], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([4, 9, 8, 5, 1, 3, 7, 1], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([5, 7, 4, 9, 1, 7, 0, 5], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 6, 8, 0, 9, 1, 4, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([9, 9, 9, 1, 1, 6, 2, 3], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([3, 8, 3, 3, 3, 9, 4, 2], device='cuda:0')
torch.Size([8, 3, 32, 32])
tensor([2, 5, 4, 6, 1, 5, 3, 0], device='cuda:0')
torch.Size([8, 3, 32, 32])

In [10]:
# Test
correct = 0
total = 0
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test inputs: %d %%' % (100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(8):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the 10000 test inputs: 54 %
Accuracy of plane : 64 %
Accuracy of   car : 84 %
Accuracy of  bird : 29 %
Accuracy of   cat : 36 %
Accuracy of  deer : 62 %
Accuracy of   dog : 42 %
Accuracy of  frog : 67 %
Accuracy of horse : 71 %
Accuracy of  ship : 44 %
Accuracy of truck : 38 %
